In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np

## Profile Report

In [ ]:
# Import all datasets
kadaster = pd.read_csv("kadaster/kadaster_merged_dataset_corrected.csv", dtype=str)
sbi = pd.read_csv("kvk/sbi_codes/kvk_sbi_codes.csv", dtype=object)
kvk = pd.read_excel("kvk/DatasetKvK01102019.xls", dtype=object)
niwo = pd.read_csv("niwo/niwo_dict.csv", dtype=object)
fenex = pd.read_csv("fenex/fenex.csv", dtype=object)
company_info = pd.read_csv("company_info/kvk_companyinfo.csv", dtype=object)

In [ ]:
# Drop columns not used for analysis
kvk_drop = ['RGL','STRVA', 'PCPLVA', 'STRCA', 'PCPLCA','HN1X30', 'HN1X2X30', 'HN2X2X30', \
            'PCCA', 'GEMK_VA', 'GEMK_CA', 'GEMNAAM', 'PROV', 'STRAAT_CA', 'HUISNR_CA', \
            'TOEV_CA', 'PC_CA', 'PLAATS_CA', 'NMI', 'HFD_N_VEST', 'C_HOOFDACT', 'VERK_ACT', \
            'NEVEN1', 'NEVEN2', 'IND_IM_EXP', 'BEHKN', 'GEOKN']
kadaster_drop = ['nrAanduiding','vblObject']

kvk = kvk.drop(kvk_drop, axis=1)
kadaster = kadaster.drop(kadaster_drop, axis=1)

In [ ]:
# Redfine data types
kvk["HUISNR_VA"] = kvk["HUISNR_VA"].astype('int64')
kadaster["huisnummer"] = kadaster["huisnummer"].astype('int64')

In [ ]:
# Make list of companies in Niwo/Fenex
niwo_kvknr = niwo["kvk_nummer"].to_list()
fenex_compname = [x.lower().strip() for x in fenex["Company Name"].to_list()]

In [ ]:
# Merge data
merged_data = pd.merge(left=kvk, right=kadaster, how='left', left_on=["PCVA","HUISNR_VA"], \
                        right_on=["postcode","huisnummer"])
merged_data = pd.merge(left=merged_data, right=sbi, how='left', on=["DOSSIER","VGNUMMER"])
merged_data = pd.merge(left=merged_data, right=company_info, how='left', left_on="DOSSIER", right_on="kvknr")

In [ ]:
# Label companies in niwo/fenex
merged_data['inNiwo'] = [1 if x in niwo_kvknr else 0 for x in merged_data['DOSSIER']] 
merged_data["inFenex"] = [1 if x.lower() in fenex_compname else 0 for x in merged_data["HN45"].astype(str)]

In [ ]:
# Drop duplicate/unneeded columns
drop_cols = ["huisnummer","postcode","kvknr"]
merged_data = merged_data.drop(drop_cols,axis=1)

In [ ]:
# Refine data types
merged_data["RV"] = merged_data["RV"].astype(int)
merged_data["oppervlakte"] = merged_data["oppervlakte"].astype(float)
merged_data["emp_class"] = merged_data["emp_class"].replace(to_replace=[np.nan,'io'],value=None)
merged_data["emp_class"] = merged_data["emp_class"].astype(int)
merged_data["reg_date"] = merged_data["reg_date"].replace(to_replace=['ull,"emplo','(w,d,s,l,i'], value=None)
merged_data['reg_date'] = pd.to_datetime(merged_data['reg_date'])

In [ ]:
# Replace values
merged_data["inOnderzoek"] = merged_data["inOnderzoek"].replace(to_replace='True', value = 1)
merged_data["inOnderzoek"] = merged_data["inOnderzoek"].replace(to_replace='False', value = 0)
merged_data["vbloStatus"] = merged_data["vbloStatus"].replace(to_replace='in gebruik (niet ingemeten)', value='in gebruik')

In [ ]:
# Filter on one branch
transport_storage = merged_data[merged_data["C_HOOFDACT"]=="H"]

In [ ]:
# Create profile report
profile = pandas_profiling.ProfileReport(transport_storage, title='Expeditors Profiling Report')

In [ ]:
profile

In [ ]:
# Safe profile report
profile.to_file(output_file="your_report.html")

## Detail Histograms

In [ ]:
# Create histogram of employee class (shift to correct ticks)
employee = kvk_kad_sbi_comp["emp_class"] +0.5
plt.hist(employee,bins=[1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5])
plt.xlabel("employee class")
plt.ylabel("number of companies")
plt.savefig("emp_class.svg")
plt.show()

In [ ]:
# Resize for readability in report
matplotlib.rc('xtick', labelsize=25) 
matplotlib.rc('ytick', labelsize=25) 
labelfont = 25

In [ ]:
# Create histogram of area (all data) (shift to correct ticks)
area = kvk_kad_sbi_comp["oppervlakte"] + 0.5
bins = 100
plt.figure(figsize=(20,5))
plt.hist(area,bins=bins)
plt.xlabel("surface area (m2)", fontsize=labelfont)
plt.ylabel("number of companies", fontsize=labelfont)
plt.savefig("area_large.svg")
plt.plot()

In [ ]:
# Create histogram of area (partial data) (shift to correct ticks)
area_small = area[area < 50000]
bins_small = 50
plt.figure(figsize=(20,5))
plt.hist(area_small,bins=bins_small)
plt.xlabel("surface area (m2)", fontsize=labelfont)
plt.ylabel("number of companies", fontsize=labelfont)
plt.savefig("area_small.svg")
plt.plot()